# Traveling salesman problem

All code for this example is based on code by Laurent Lessard: https://laurentlessard.com.

Find the closed tour that visits all ten cities and has the minimum total length. Distances between pairs of cities are given in the table below:

|         |  ATL  |  ORD  |  DEN  |  IAH  |  LAX  |  MIA  |  JFK  |  SFO  |  SEA  |  DCA  |
|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|**ATL**  |    0  |  587  | 1212  |  701  | 1936  |  604  |  748  | 2139  | 2182  |  543  |
|**ORD**  |  587  |    0  |  920  |  940  | 1745  | 1188  |  713  | 1858  | 1737  |  597  |
|**DEN**  | 1212  |  920  |    0  |  879  |  831  | 1726  | 1631  |  949  | 1021  | 1494  |
|**IAH**  |  701  |  940  |  879  |    0  | 1379  |  968  | 1420  | 1645  | 1891  | 1220  |
|**LAX**  | 1936  | 1745  |  831  | 1379  |    0  | 2339  | 2451  |  347  |  959  | 2300  |
|**MIA**  |  604  | 1188  | 1726  |  968  | 2339  |    0  | 1092  | 2594  | 2734  |  923  |
|**JFK**  |  748  |  713  | 1631  | 1420  | 2451  | 1092  |    0  | 2571  | 2408  |  205  |
|**SFO**  | 2139  | 1858  |  949  | 1645  |  347  | 2594  | 2571  |    0  |  678  | 2442  |
|**SEA**  | 2182  | 1737  | 1021  | 1891  |  959  | 2734  | 2408  |  678  |    0  | 2329  |
|**DCA**  |  543  |  597  | 1494  | 1220  | 2300  |  923  |  205  | 2442  | 2329  |    0  |

In [9]:
# Define the problem data (cities and costs)
# Run this, then run the final block of this notebook to load the helper functions before continuing
using JuMP, NamedArrays, HiGHS

cities = [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]

distances = [     0   587  1212   701  1936   604   748  2139  2182   543
                587     0   920   940  1745  1188   713  1858  1737   597
               1212   920     0   879   831  1726  1631   949  1021  1494
                701   940   879     0  1379   968  1420  1645  1891  1220
               1936  1745   831  1379     0  2339  2451   347   959  2300
                604  1188  1726   968  2339     0  1092  2594  2734   923
                748   713  1631  1420  2451  1092     0  2571  2408   205
               2139  1858   949  1645   347  2594  2571     0   678  2442
               2182  1737  1021  1891   959  2734  2408   678     0  2329
                543   597  1494  1220  2300   923   205  2442  2329     0  ]

c = NamedArray(distances,(cities,cities))
N = size(distances,1);

In [ ]:
# Run the final block of this notebook to load helper functions before continuing!
mapSolution(0)
xlabel("plot showing the 10 airport locations");
savefig("tsp_initial.pdf")

## Solve TSP using min-cost flow relaxation

In [ ]:
# solve the simplest version (min-cost flow version; it's an assignment problem)
#using Pkg
#Pkg.add("Cbc")

m = Model(HiGHS.Optimizer)
@variable(m, 0 <= x[cities,cities] <= 1)                                 # LP relaxation
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)    # exacly one edge out of each node
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)    # exactly one edge into each node
@constraint(m, c3[i in cities], x[i,i] == 0 )                       # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))  # minimize total cost
optimize!(m)

# pretty print the solution
xx = value.(x)
sol = NamedArray(zeros(Int,N,N),(cities,cities))
for i in cities
    for j in cities
        sol[i,j] = Int(xx[i,j])
    end
end
println(sol)
sleep(1)
display(getAllSubtours(sol))
mapSolution(sol)
xlabel("looks like the relaxation isn't what we wanted!")
tight_layout()
savefig("tsp_sol_LP.pdf")
;

## Solve TSP using adaptive subtour elimination

In [ ]:
m = Model(HiGHS.Optimizer)
@variable(m, x[cities,cities], Bin)                                       # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)     # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)     # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                        # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost

sols = []

for iters = 1:30
    optimize!(m)
    println("Tour length: ", objective_value(m))
    xx = value.(x)
    push!(sols,xx)
    subtours = getAllSubtours(xx)  # get all the subtours
    display(subtours)
    sleep(1)
    len = length(subtours)
    if len == 1                    # solution is just a single tour!
        println("SOLVED!")
        break
    else
        for subtour in subtours
            L = length(subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end

5-element Vector{Any}:
 [:ATL, :MIA, :ATL]
 [:ORD, :IAH, :ORD]
 [:DEN, :SEA, :DEN]
 [:LAX, :SFO, :LAX]
 [:JFK, :DCA, :JFK]

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
20 rows, 90 cols, 180 nonzeros
20 rows, 90 cols, 180 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   20 rows
   90 cols (90 binary, 0 integer, 0 implied int., 0 continuous)
   180 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 T       0       0         0   0.00%   0               6234             100.00%        0      0      0        21     0.0s

Solving report
  Status            Optimal
  Primal bound      6234
  Dual bound        6234
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    6234 (objective)

3-element Vector{Any}:
 [:ATL, :IAH, :MIA, :ATL]
 [:ORD, :DCA, :JFK, :ORD]
 [:DEN, :LAX, :SFO, :SEA, :DEN]

In [ ]:
# plot the iterations as we eliminate subtours
figure(figsize=(13,6))
for i = 1:6
    subplot(2,3,i)
    mapSolution(sols[i])
    xlabel(string("iteration ",i))
end
tight_layout()
savefig("tsp_sol_elim.pdf")

## Miller-Tucker-Zemlin formulation

In [ ]:
m = Model(HiGHS.Optimizer)
@variable(m, x[cities,cities], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)      # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)      # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost
                                    
# MTZ variables and constraints
@variable(m, u[cities])
@constraint(m, c4[i in cities, j in cities[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx)  # get cycle containing Atlanta
display(subtours)
println("Tour length: ", objective_value(m))

In [ ]:
mapSolution(xx)

## Helper functions

In [ ]:
#Run this to install everything the first time through!!!
#Will fail if you do not install everything first!
#using Pkg;
#Pkg.add("PyPlot")
#Pkg.add("PythonCall")
#Pkg.add("Conda")
#using CondaPkg
#CondaPkg.add("basemap")

# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in cities
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = cities
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end

# HELPER FUNCTION FOR MAPPING AIRPORTS

data = [ 33.636700  -84.427863 
         41.977320  -87.908005
         39.861667 -104.673166
         29.645417  -95.278888
         33.942437 -118.408121
         25.795361  -80.290110
         40.639926  -73.778694
         37.618806 -122.375416
         47.449889 -122.311777
         38.851444  -77.037721 ]
lat = Dict(zip(cities,data[:,1]))
lon = Dict(zip(cities,data[:,2]))

using PyPlot, PyCall
basemap = pyimport("mpl_toolkits.basemap")

function mapSolution(x=0)
    m=basemap.Basemap(projection="merc", resolution="l",llcrnrlat=23,llcrnrlon=-126,urcrnrlat=50,urcrnrlon=-70)
    m[:drawmapboundary](fill_color="#4771a5")
    m[:fillcontinents](color="#555555")

    # plot airports
    for i in cities
        m[:plot](lon[i], lat[i], "ro" ,latlon=true)
    end

    # plot tours
    if x==0
        return
    else
        tours = getAllSubtours(x)
        for t in tours
            L = length(t)-1
            for i in 1:L
                m[:drawgreatcircle](lon[t[i]],lat[t[i]],lon[t[i+1]],lat[t[i+1]],linewidth=1,color="b")
            end
        end
    end
end
;